In [1]:
import openai
import os
import pandas as pd

In [5]:
# get string of the key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
os.getenv("OPENAI_API_KEY")[0:3]

'sk-'

In [7]:
df = pd.read_csv('sales_data_sample.csv')
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME
0,10107,30,95.70,2871.00,2/24/2003 0:00,1,2,2003,Motorcycles,2125557818,897 Long Airport Avenue,NYC,NY,10022,USA,Yu,Kwai
1,10121,34,81.35,2765.90,5/7/2003 0:00,2,5,2003,Motorcycles,26.47.1555,59 rue de l'Abbaye,Reims,NaN,51100,France,Henriot,Paul
2,10134,41,94.74,3884.34,7/1/2003 0:00,3,7,2003,Motorcycles,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,NaN,75508,France,Da Cunha,Daniel
3,10145,45,83.26,3746.70,8/25/2003 0:00,3,8,2003,Motorcycles,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,USA,Young,Julie
4,10159,49,100.00,5205.27,10/10/2003 0:00,4,10,2003,Motorcycles,6505551386,7734 Strong St.,San Francisco,CA,NaN,USA,Brown,Julie


In [8]:
df.groupby("QTR_ID").sum()['SALES']

C:\Users\manor\AppData\Local\Temp\ipykernel_6108\2501432.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby("QTR_ID").sum()['SALES']


QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64

In [10]:
# What was the total sum of sales per quarter?

In [11]:
from sqlalchemy import create_engine
from sqlalchemy import text

TEMP db in RAM

Push pandas df --> TEMP DB

SQL query on TEMP DB

In [12]:
temp_db = create_engine('sqlite:///:memory:',echo=True)

In [13]:
data = df.to_sql(name='Sales', con=temp_db)

2023-03-21 21:11:19,964 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-21 21:11:19,965 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-03-21 21:11:19,966 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-21 21:11:19,968 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-03-21 21:11:19,971 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-21 21:11:19,972 INFO sqlalchemy.engine.Engine ROLLBACK
2023-03-21 21:11:19,976 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-21 21:11:19,979 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT
)


2023-03-21 21:11:19,980 INFO sqlalche

In [14]:
with temp_db.connect() as conn:
    # makes the connection and we can run code within and automatically close conn
    result = conn.execute(text("SELECT * FROM Sales"))

2023-03-21 21:13:59,075 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-21 21:13:59,077 INFO sqlalchemy.engine.Engine SELECT * FROM Sales
2023-03-21 21:13:59,078 INFO sqlalchemy.engine.Engine [generated in 0.00357s] ()
2023-03-21 21:13:59,079 INFO sqlalchemy.engine.Engine ROLLBACK


In [15]:
result.all()

[(0, 10107, 30, 95.7, 2871.0, '2/24/2003 0:00', 1, 2, 2003, 'Motorcycles', '2125557818', '897 Long Airport Avenue', 'NYC', 'NY', '10022', 'USA', 'Yu', 'Kwai'),
 (1, 10121, 34, 81.35, 2765.9, '5/7/2003 0:00', 2, 5, 2003, 'Motorcycles', '26.47.1555', "59 rue de l'Abbaye", 'Reims', None, '51100', 'France', 'Henriot', 'Paul'),
 (2, 10134, 41, 94.74, 3884.34, '7/1/2003 0:00', 3, 7, 2003, 'Motorcycles', '+33 1 46 62 7555', '27 rue du Colonel Pierre Avia', 'Paris', None, '75508', 'France', 'Da Cunha', 'Daniel'),
 (3, 10145, 45, 83.26, 3746.7, '8/25/2003 0:00', 3, 8, 2003, 'Motorcycles', '6265557265', '78934 Hillside Dr.', 'Pasadena', 'CA', '90003', 'USA', 'Young', 'Julie'),
 (4, 10159, 49, 100.0, 5205.27, '10/10/2003 0:00', 4, 10, 2003, 'Motorcycles', '6505551386', '7734 Strong St.', 'San Francisco', 'CA', None, 'USA', 'Brown', 'Julie'),
 (5, 10168, 36, 96.66, 3479.76, '10/28/2003 0:00', 4, 10, 2003, 'Motorcycles', '6505556809', '9408 Furth Circle', 'Burlingame', 'CA', '94217', 'USA', 'Hirano

In [16]:
with temp_db.connect() as conn:
    # makes the connection and we can run code within and automatically close conn
    result = conn.execute(text("SELECT SUM(SALES) FROM Sales"))

2023-03-21 21:14:40,518 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-21 21:14:40,519 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales
2023-03-21 21:14:40,520 INFO sqlalchemy.engine.Engine [generated in 0.00250s] ()
2023-03-21 21:14:40,526 INFO sqlalchemy.engine.Engine ROLLBACK


In [18]:
result.all()

[(10032628.85000001,)]

In [20]:
def create_table_definition(df):
    # setup prompt
    prompt = """### sqlite SQL table, with it properties:
    #
    # Sales({})
    #
    """.format(",".join(str(col) for col in df.columns))

    return prompt

In [22]:
print(create_table_definition(df))

### sqlite SQL table, with it properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #
    


In [25]:
def prompt_input():
    nlp_text = input("Enter the info you want: ")
    return nlp_text

In [26]:
prompt_input()

'grab all sales per quater'

In [27]:
# combine results into single function

def combine_prompts(df,query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition + query_init_string

In [28]:
nlp_text = prompt_input()   #NLP
combine_prompts(df, nlp_text)   # DF + query that does.... + NLP

'### sqlite SQL table, with it properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)\n    #\n    ### A query to answer: query to answer total sales per quarter\nSELECT'

OpenAI Completion Call Parameters

- Model

- Prompt

- Temperature

- Max Tokens

- Top P

- N

- Frequency Penalty

- Presence Penalty

In [29]:
response = openai.Completion.create(
    model = 'code-davinci-002',
    prompt = combine_prompts(df,nlp_text),
    temperature = 0,
    max_tokens = 150,
    top_p = 1.0,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = ['#',';']
)

In [30]:
response

<OpenAIObject text_completion id=cmpl-6wi5c6m0Gq88qR9DK8E4j45LORv6V at 0x21aa5133f40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " QTR_ID, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY QTR_ID\nORDER BY QTR_ID"
    }
  ],
  "created": 1679450524,
  "id": "cmpl-6wi5c6m0Gq88qR9DK8E4j45LORv6V",
  "model": "code-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 36,
    "prompt_tokens": 119,
    "total_tokens": 155
  }
}

In [31]:
response['choices'][0]['text']

' QTR_ID, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY QTR_ID\nORDER BY QTR_ID'

In [32]:
def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(" "):
        query = "SELECT" + query
    return query

In [33]:
handle_response(response)

'SELECT QTR_ID, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY QTR_ID\nORDER BY QTR_ID'

In [34]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-03-21 22:06:02,198 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-21 22:06:02,200 INFO sqlalchemy.engine.Engine SELECT QTR_ID, SUM(SALES) AS TOTAL_SALES
FROM Sales
GROUP BY QTR_ID
ORDER BY QTR_ID
2023-03-21 22:06:02,200 INFO sqlalchemy.engine.Engine [generated in 0.00208s] ()
2023-03-21 22:06:02,203 INFO sqlalchemy.engine.Engine ROLLBACK


In [35]:
result.all()

[(1, 2350817.7300000004),
 (2, 2048120.2999999986),
 (3, 1758910.8099999994),
 (4, 3874780.01)]